In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import csv
import os
import argparse
import codecs
import json
import requests

# cd to the path of the raw dataset
%cd /content/drive/MyDrive/dataset

/content/drive/MyDrive/dataset


In [ ]:
def diacritize(text):
  url = 'https://farasa.qcri.org/webapi/seq2seq_diacritize/'
  api_key = # your API key
  dialect = "mor"
  payload = {'text': text, 'api_key': api_key, "dialect": dialect}
  data = requests.post(url, data=payload)
  diacritized = json.loads(data.text)
  return diacritized 

In [ ]:
# to not lose what has been diacritized when the diacritizer crashes (when an encountering a character that is not expected)
rows = []

In [ ]:
# diacritize
i = 1
with open('lefttodiacritize3.csv', 'r', encoding="utf-8") as readFile:
  reader = csv.reader(readFile)
  for row in reader:
    if (i >= 427 and row[0] != "num"):
      # removing repeating characters
      row[2] = re.sub(r'(.)\1+', r'\1\1', row[2])

      # dealing with ila hamza
      pattern = "\u0627\u0644\u0649 "
      repl = "\u0625\u0644\u0649 "
      row[2] = re.sub(pattern, repl, row[2])

      # removing punctuation marks
      row[2] = re.sub(r'[؟!؛،.ـ():-_{}]', ' ', row[2])
      # print("row2 ", row[2])
      result = diacritize(row[2])
      # print("result ", result["text"])

      row[2] = result["text"]
      rows.append(row)
      # print("i ", i)
    i += 1


In [ ]:
# to keep track where the diacritizer crashed so that you can start from there and avoid starting from the beginning by changing the condition of i in the previous cell
print(i)

522


In [ ]:
with open("part3diacritized.csv", "w", encoding="utf-8") as writeFile:
  writer = csv.writer(writeFile)
  writer.writerows(rows)

In [ ]:
def transliterate(text, abjad):
  transliterated = ""
  for char in text: 
    if char in abjad:
      transliterated += str(abjad[char])
    else:
      transliterated += str(char)
  return transliterated

In [ ]:
abjad = {u"\u0622":'|', u"\u0627":'A',
         u"\u0628":'b', u"\u062A":'t', u"\u062B":'v', u"\u062C":'j',
         u"\u062D":'H', u"\u062E":'x', u"\u062F":'d', u"\u0630":'*', 
         u"\u0631":'r', u"\u0632":'z', u"\u0633":'s', u"\u0634":'$', 
         u"\u0635":'S', u"\u0636":'D', u"\u0637":'T', u"\u0638":'Z', 
         u"\u0639":'E', u"\u063A":'g', u"\u0641":'f', u"\u0642":'q', 
         u"\u0643":'k', u"\u0644":'l', u"\u0645":'m', u"\u0646":'n',
         u"\u0647":'h', u"\u0648":'w', u"\u0649":'y', u"\u064A":'y', 
         u"\u0629":'p', u"\u0621":'\'', u"\u0623":'>', u"\u0625":'<', 
         u"\u0624":'&', u"\u0626":'}', u"\u0654":'\'', u"\u0655":'\'', 
         u"\u064B":'F', u"\u064C":'N', u"\u064D":'K', u"\u064E":'a', 
         u"\u064F":'u', u"\u0650":'i', u"\u0651":'~', u"\u0652":'o'}

In [ ]:
# transliteration
j = 1
csv_rows = []
with open('FirstTrain.csv', 'r', encoding="utf-8") as readFile:
  reader = csv.reader(readFile)
  for row in reader:
    if (j >= 1 and row[0] != "num"):
      filename = row[3].split(".")[0]

      row[2] = transliterate(row[2], abjad)

      with open(f"lab/{filename}.lab", "w") as writeFile:
        writeFile.write(row[2])
        j+=1
      
      csv_rows.append(row)


with open("transliterated.csv", "w", encoding="utf-8") as writeFile:
  writer = csv.writer(writeFile)
  writer.writerows(csv_rows)